In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import intel_extension_for_pytorch as ipex
import os
import transformers

os.environ["TOKENIZERS_PARALLELISM"] = "true"

model_hf = "mistralai/Mistral-7B-v0.1"
device = torch.device("xpu") # the device to load the model onto
torch_dtype = torch.bfloat16
xpu_autocast = torch.xpu.amp.autocast
cpu_autocast = torch.cpu.amp.autocast

tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    trust_remote_code=True
)

model = (
    AutoModelForCausalLM.from_pretrained(
        "mistralai/Mistral-7B-v0.1",
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        torch_dtype=torch_dtype
    ).to(device).eval()
)

model = ipex.optimize_transformers(model, dtype=torch_dtype)

# chat_template = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]

# encodeds = tokenizer.apply_chat_template(chat_template, return_tensors="pt")


# model_inputs = encodeds.to(device)

# with xpu_autocast(enabled=True,dtype=torch_dtype):
#     with torch.no_grad():
#         generated_ids = model.generate(
#             model_inputs, 
#             max_new_tokens=1000, 
#             do_sample=True
#         )
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/intel/oneapi/intelpython/latest/envs/pytorch-gpu/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:611: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn(
/opt/intel/oneapi/intelpython/latest/envs/pytorch-gpu/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:618: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(


In [26]:
input_text = "How old is a giraffe?"
# input_text "Today I went to the park and "
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
with xpu_autocast(enabled=True, dtype=torch_dtype):
    output = model.generate(input_ids, max_length=50,pad_token_id=tokenizer.eos_token_id)
# print(tokenizer.decode(output[0], skip_special_tokens=True))
print(tokenizer.batch_decode(output[:, input_ids.shape[1]:],skip_special_tokens=True)[0])



Giraffes are the tallest mammals in the world. They can grow up to 18 feet tall and weigh up to 3,000 pounds.

Gir




Giraffes are the tallest mammals in the world. They can grow up to 18 feet tall and weigh up to 3,000 pounds.

Gir


In [18]:
output[1]

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [7]:
tokenizer.default_chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif true == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must 

In [3]:
tokenizer.default_chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif true == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must 

In [ ]:
class ChatBotModel:
    """
    ChatBotModel is a class for generating responses based on text prompts using a pretrained model. 
    """

    def __init__(
        self,
        model_id_or_path,
        torch_dtype

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

What is your favourite condiment? [/INST]
<</IINST>>
I love using ketchup to add some extra flavour to my meals.
<</IINST>>

What's your favorite thing about [NAME]? [/INST]
<</IINST>>
I have always enjoyed [NAME]'s positivity and honesty. You are always willing to help others!
<</IINST>>

What's your favorite restaurant? [/INST]
<</IINST>>
I personally love to dine at a great steak house like Ruth's Chris Steak House.
<</IINST>>

What's your favorite color? 

In [ ]:
generated_ids = model.generate(
    model_inputs, 
    max_new_tokens=1000, 
    do_sample=True
)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [ ]:
import torch
import intel_extension_for_pytorch as ipex
import random
import os
# from vllm import SamplingParams

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# from transformers import LlamaTokenizer, LlamaForCausalLM
import transformers

In [ ]:
transformers.__version__

In [ ]:
# pip install -U transformers

In [ ]:
# random seed
if torch.xpu.is_available():
    seed = 88
    random.seed(seed)
    torch.xpu.manual_seed(seed)
    torch.xpu.manual_seed_all(seed)

In [ ]:
model_hf = "mistralai/Mistral-7B-v0.1"
# model_hf = "mistralai/Mistral-7B-Instruct-v0.1"
torch_dtype = torch.bfloat16
device = torch.device("xpu")

In [ ]:
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained(model_hf)
tokenizer = AutoTokenizer.from_pretrained(model_hf)

In [ ]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")



In [ ]:
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

## Disclaimer
To the extent that any public or non-Intel datasets or models are referenced by or accessed using these materials those datasets or models are provided by the third party indicated as the content source. Intel does not create the content and does not warrant its accuracy or quality. By accessing the public content, or using materials trained on or with such content, you agree to the terms associated with that content and that your use complies with the applicable license.

Intel expressly disclaims the accuracy, adequacy, or completeness of any such public content, and is not liable for any errors, omissions, or defects in the content, or for any reliance on the content. Intel is not liable for any liability or damages relating to your use of public content.

Intel’s provision of these resources does not expand or otherwise alter Intel’s applicable published warranties or warranty disclaimers for Intel products or solutions, and no additional obligations, indemnifications, or liabilities arise from Intel providing such resources. Intel reserves the right, without notice, to make corrections, enhancements, improvements, and other changes to its materials.
